In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets

데이터 전처리  
우리가 실제로 접하게 되는 데이터는 사이킷런에서 제공하는 데이터와는 다르게 머신러닝을 적용하기 전에 데이터를 가공해야 하는 경우가 많다. 이를 데이터 전처리, 피쳐 엔지니어링 등으로 부른다.

결측치(missing value) 처리  
데이터셋의 일부에 데이터가 존재하지 않을 경우 결측치라고 부르며, 머신러닝 알고리즘을 적용하기 전에 미리 결측치를 처리하는 것이 중요하다.

In [2]:
# 데이터프레임에 데이터가 존재하지 않는 결측치는 없지만 부적절한 값(이상치)이 존재한다.
# 나이에 1000이라는 값이 존재하는 것과 나머지 피쳐에 unknown이라는 값이 입력된 부적절한 값은 결측치로 처리하는 것이 좋다.
df = pd.DataFrame([
    [42, 'male', 12, 'reading', 'class2'],
    [35, 'unknown', 3, 'cooking', 'class1'],
    [1000, 'female', 7, 'cycling', 'class3'],
    [1000, 'unknown', 21, 'unknown', 'unknown']
])
# 나이, 성별, 태어난 달, 취미, 타겟
df.columns = ['age', 'gender', 'month_birth', 'hobby', 'target']
df

,age,gender,month_birth,hobby,target
0,42,male,12,reading,class2
1,35,unknown,3,cooking,class1
2,1000,female,7,cycling,class3
3,1000,unknown,21,unknown,unknown


In [3]:
# unique() 메소드로 각 열의 유일한 값을 확인할 수 있다.
print('age:', df.age.unique())
print('gender:', df.gender.unique())
print('month_birth:', df.month_birth.unique())
print('hobby:', df.hobby.unique())
print('target:', df.target.unique())

age: [  42   35 1000]
gender: ['male' 'unknown' 'female']
month_birth: [12  3  7 21]
hobby: ['reading' 'cooking' 'cycling' 'unknown']
target: ['class2' 'class1' 'class3' 'unknown']


In [4]:
# 부적절한 데이터값을 결측치로 바꾸기 위해서 np.nan을 사용한다.
df.loc[df.age > 150, ['age']] = np.nan
df.loc[df.gender == 'unknown', ['gender']] = np.nan
df.loc[df.month_birth > 12, ['month_birth']] = np.nan
df.loc[df.hobby == 'unknown', ['hobby']] = np.nan
df.loc[df.target == 'unknown', ['target']] = np.nan
df

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3
3,NaN,NaN,NaN,NaN,NaN


In [5]:
# 데이터셋 각 열의 결측치 개수를 확인한다.
df.isnull().sum() # df.isna().sum()

age            2
gender         2
month_birth    1
hobby          1
target         1
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          2 non-null      float64
 1   gender       2 non-null      object 
 2   month_birth  3 non-null      float64
 3   hobby        3 non-null      object 
 4   target       3 non-null      object 
dtypes: float64(2), object(3)
memory usage: 288.0+ bytes


결측치를 처리하는 데 주로 쓰이는 방법은 결측치를 삭제하거나 다른 값으로 변경하는 방법이 있다.

In [7]:
# dropna() 메소드로 결측치를 삭제할 수 있다.
# axis 속성을 생략하거나 axis=0으로 지정하면 결측치가 하나라도 포함된 모든 행을 삭제한다.
# inplace 속성을 inplace=True로 지정하면 실행 결과를 즉시 데이터프레임에 반영한다.
df2 = df.dropna(axis=0)
df2

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2


In [8]:
# axis 속성을 axis=1로 지정하면 결측치가 하나라도 포함된 모든 열을 삭제한다.
df3 = df.dropna(axis=1)
df3

""
0
1
2
3


In [9]:
# how 속성을 how='all'로 지정하면 행의 값이 모두 결측치인 행을 삭제한다.
df4 = df.dropna(how='all')
df4

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [10]:
# thresh 속성을 thresh=n으로 지정하면 결측치를 제외한 값의 개수가 n보다 작은 행을 삭제한다.
df5 = df.dropna(thresh=1)
df5

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,NaN,3.0,cooking,class1
2,NaN,female,7.0,cycling,class3


In [11]:
# subset 속성을 subset=['열이름']으로 지정하면 특정 열에 결측치가 존재하는 행을 삭제한다.
df6 = df.dropna(subset=['gender'])
df6

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
2,NaN,female,7.0,cycling,class3


In [12]:
# fillna() 메소드로 결측치를 대체할 값을 지정할 수 있다.
df7 = df.fillna(0)
df7

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,0,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,0,0.0,0,0


In [13]:
# value 속성으로 결측치를 대체할 값이 지정된 딕셔너리를 지정하면 열별로 결측치를 대체할 수 있다.
# 열 이름을 key로 하고 결측치를 대체할 값을 value로 지정한 딕셔너리
alter_values = {'age': 0, 'gender': 'U', 'month_birth': 0, 'hobby': 'U', 'target': 'class4'}
df8 = df.fillna(value=alter_values)
df8

,age,gender,month_birth,hobby,target
0,42.0,male,12.0,reading,class2
1,35.0,U,3.0,cooking,class1
2,0.0,female,7.0,cycling,class3
3,0.0,U,0.0,U,class4


기계 처리에 적합하도록 클래스 레이블 설정  
target 열을 보면 문자열(string) 형태인데, 데이터 전처리 과정에서 문자열을 정수(int)로 바꿔야 하는 경우가 있다.

사이킷런을 사용하지 않고 레이블 설정하기

In [14]:
df9 = df8

In [15]:
# 라벨링할 타겟만 얻어와서 오름차순으로 정렬한다.
y_arr = df9.target.values
# print(type(y_arr))
y_arr.sort() # 오름차순
# y_arr = y_arr[::-1] # 역순으로 뒤집어서 다시 저장한다.
y_arr

array(['class1', 'class2', 'class3', 'class4'], dtype=object)

In [16]:
# 반복문을 실행해서 정렬된 타겟 데이터를 key로 일련번호를 value로 하는 딕셔너리를 만든다.
dict_y = {}
for value, key in enumerate(y_arr):
    # print(value, key)
    dict_y[key] = value
dict_y

{'class1': 0, 'class2': 1, 'class3': 2, 'class4': 3}

In [17]:
# replace() 메소드를 이용해서 딕셔너리에 저장된 데이터로 타겟을 라벨링 한다.
# df9['target'].replace(dict_y, inplace=True)
df9['target_1'] = df9.target.replace(dict_y)
df9

,age,gender,month_birth,hobby,target,target_1
0,42.0,male,12.0,reading,class1,0
1,35.0,U,3.0,cooking,class2,1
2,0.0,female,7.0,cycling,class3,2
3,0.0,U,0.0,U,class4,3


사이킷런을 사용해서 레이블 설정하기

In [18]:
# 클래스 라벨링을 하기 위해 import 한다.
from sklearn.preprocessing import LabelEncoder

In [19]:
df10 = df8
class_label = LabelEncoder() # 클래스 라벨링을 실행하는 객체를 생성한다.
data_value = df10.target.values # 라벨링을 실행할 데이터를 얻어온다.
print(data_value)
# fit_transform() 메소드로 라벨링할 데이터를 넘겨서 라벨링을 실행한다.
y_new = class_label.fit_transform(data_value)
y_new

['class1' 'class2' 'class3' 'class4']


array([0, 1, 2, 3])

In [20]:
# 라벨링한 결과를 데이터프레임에 넣어준다.
df10['target_2'] = y_new
df10

,age,gender,month_birth,hobby,target,target_1,target_2
0,42.0,male,12.0,reading,class1,0,0
1,35.0,U,3.0,cooking,class2,1,1
2,0.0,female,7.0,cycling,class3,2,2
3,0.0,U,0.0,U,class4,3,3


원-핫 인코딩(one-hot encoding)  
원-핫 인코딩은 클래스를 라벨링하는 또 다른 방법으로 오직 0과 1만 사용한 벡터를 이용해서 데이터 값을 나타내는 것이다.

판다스를 이용한 원-핫 인코딩

In [21]:
df11 = df8

In [22]:
# get_dummies() 메소드의 인수로 원-핫 인코딩을 실행한다.
df12 = pd.get_dummies(df11.target)
df12

,class1,class2,class3,class4
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [23]:
df11['target_2'] = df11.target_2.astype(str) # 타겟의 값을 문자열로 변환한다.
df13 = pd.get_dummies(df11.target_2)
df13

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1


In [24]:
# 위의 방법에서는 길이가 4인 벡터를 이용해서 클래스를 구분했다.
# drop_first 속성을 지정하면 벡터 길이를 하나 줄인 벡트럴 이용해 원-한 인코딩을 할 수 있다.
# drop_first 속성을 drop_first=True로 지정하면 3개의 0과 1의 조합으로 4개의 클래스가 구분되는 것을 확인할 수 있다.
df14 = pd.get_dummies(df11.target, drop_first=True)
df14

,class2,class3,class4
0,0,0,0
1,1,0,0
2,0,1,0
3,0,0,1


사이킷런을 이용한 원-핫 인코딩

In [25]:
# 원-핫 인코딩을 하기 위해 import 한다.
from sklearn.preprocessing import OneHotEncoder

In [26]:
one_hot_encoder = OneHotEncoder() # 원-핫 인코딩을 실행하는 객체를 생성한다.
y = df8[['target']]
# print(y)
y_hot = one_hot_encoder.fit_transform(y)
print(type(y_hot))
print(y_hot)
print(type(y_hot.toarray()))
print(y_hot.toarray())

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
<class 'numpy.ndarray'>
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


텐서플로우를 이용한 원-핫 인코딩

In [27]:
# 원-핫 인코딩을 하기 위해 import 한다.
from tensorflow.keras.utils import to_categorical

In [28]:
y = df8[['target_1']]
y_onehot = to_categorical(y)
print(type(y_onehot))
print(y_onehot)

<class 'numpy.ndarray'>
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


데이터 스케일링(data scaling)  
피쳐는 제각기 다른 단위를 가지기 때문에 이로 인해 숫자 자체의 크기 차이가 발생한다. 데이터 스케일링은 데이터 값이 단위의 영향을 받지 않도록 변형하는 것을 의미한다. 전체 데이터를 트레이닝 데이터와 테스트 데이터로 나누고 스케일링 기준이 되는 값을 구할 때는 트레이닝 데이터만으로 구한다.

표준화 스케일링(standard scaling)  
대표적인 데이터 스케일링 방법으로 평균 0, 표준 편차가 1이 되도록 변경하는 방법이다.

$$\frac{x_i - \bar x}{\sigma}$$

위 식의 의미는 기존 데이터 값에서 평균을 뺀 후 표준 편차로 나눈 값으로 바꾼다는 의미이다.

In [29]:
# 표준화 스케일링을 하기 위해서 import 한다.
from sklearn.preprocessing import StandardScaler

In [30]:
std = StandardScaler() # 표준화 스케일링 객체를 만든다.
# 표준화 스케일러를 적용할 데이터를 fit() 메소드로 먼저 학습(적합) 시킨 후 transform() 메소드로 적용시킨다.
y = df8[['month_birth']]
print(y)
std.fit(y) # 표준화 스케일러에 month_birth 열을 적합시킨다.
x_std = std.transform(y) # 적합(fit)된 표준화 스케일러를 기준으로 month_birth 열 데이터 값을 변형시킨다.
print(x_std)

   month_birth
0         12.0
1          3.0
2          7.0
3          0.0
[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [31]:
# 실제로 표준화 스케일링 후 데이터가 평균 0, 표준 편차 1인지 확인한다.
print('평균: {:.1f}, 표준편차: {:.1f}'.format(np.mean(x_std), np.std(x_std)))

평균: -0.0, 표준편차: 1.0


In [32]:
# fit() 메소드와 transform() 메소드를 각각 실행한 표준화 스케일링은 fit_transform() 메소드를 사용하면 한 번에
# 실행할 수 있다.
x_std2 = std.fit_transform(y)
print(x_std2)

[[ 1.44444444]
 [-0.55555556]
 [ 0.33333333]
 [-1.22222222]]


In [33]:
print('평균: {:.1f}, 표준편차: {:.1f}'.format(np.mean(x_std2), np.std(x_std2)))

평균: -0.0, 표준편차: 1.0


로버스트 스케일링(robust scaling)  
로버스트 스케일링은 표준화 스케일링을 변형한 방법으로 중앙값(median)과 사분위수(quantile)를 사용한다. 극단값의 영형을 거의 받지 않는다는 장점이 있다.

$$\frac{x_i - q_2}{q_3 - q_1}$$

위 식에서 $q_1$은 1사분위수, $q_2$은 2사분위수(중앙값), $q_3$은 3사분위수를 의미한다.

In [34]:
# 로버스트 스케일링을 하기 위해서 import 한다.
from sklearn.preprocessing import RobustScaler

In [35]:
robust = RobustScaler() # 로버스트 스케일링 객체를 만든다.
y = df8[['month_birth']]
robust.fit(y)
x_robust = robust.transform(y)
print(x_robust)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


In [36]:
x_robust2 = robust.fit_transform(y)
print(x_robust2)

[[ 1.16666667]
 [-0.33333333]
 [ 0.33333333]
 [-0.83333333]]


최소-최대 스케일링(min-max scaling)  
최소-최대 스케일링은 데이터 값의 범위를 최소값(0)과 최대값(1) 사이의 값으로 범위를 제한한다.

$$\frac{x_i - min(x)}{max(x) - min(x)}$$

In [37]:
# 최소-최대 스케일링을 하기 위해서 import 한다.
from sklearn.preprocessing import MinMaxScaler

In [38]:
minmax = MinMaxScaler() # 최소-최대 스케일링 객체를 만든다.
y = df8[['month_birth']]
minmax.fit(y)
x_minmax = minmax.transform(y)
print(x_minmax)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


In [39]:
x_minmax2 = minmax.fit_transform(y)
print(x_minmax2)

[[1.        ]
 [0.25      ]
 [0.58333333]
 [0.        ]]


노멀 스케일링(normalizer scaling)  
노멀 스케일링은 벡터의 유클라디안 거리가 1이 되도록 데이터 값을 변경한다.  
노멀 스케일링은 주로 벡터의 길이는 상관없고 방향(각도)만 고려할 때 사용한다.  
표준화, 로버스트, 최소-최대 스케일링은 열 기준인 것과 다르게 노멀 스케일링은 행 기준이다.

$$\frac{x_i}{\sqrt{x_i^2 + y_i^2 + z_i^2}}$$

In [40]:
# 노멀 스케일링을 하기 위해서 import 한다.
from sklearn.preprocessing import Normalizer

In [41]:
normal = Normalizer() # 최소-최대 스케일링 객체를 만든다.
y = df8[['age', 'month_birth']]
normal.fit(y)
x_normal = normal.transform(y)
print(x_normal)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


In [42]:
x_normal2 = normal.fit_transform(y)
print(x_normal2)

[[0.96152395 0.27472113]
 [0.99634665 0.08540114]
 [0.         1.        ]
 [0.         0.        ]]


데이터 스케일링 과정에서 fit() 메소드는 트레이닝 데이터셋에 대해서만 사용하며, 테스트 데이터셋에는 fit() 메소드를 사용하지 않고 transform() 메소드만 사용한다. 그 이유는 데이터 스케일러는 트레이닝 데이터를 기반으로 만들기 때문이다.

from sklearn.preprocessing import StandardScaler

stand_scale = StandardScaler()  
x_train_std = stand_scale.fit_transform(x_train) # 트레이닝 데이터셋  
x_test_std = stand_scale.transform(x_test) # 테스트 데이터셋

만약에 stand_scale.transform(x_test)이 아닌 stand_scale.fit_transform(x_test)를 사용하면 테스트 데이터의 평균, 표준 편차를 이용해 스케일링을 적용하므로 트레이닝 데이터를 이용해 생성했던 스케일링 범위 및 파이리터가 달라진다.